In [1]:
in_name = "manuscript_covariates_5_final"
out_name="table_1_final"

## Loading Dataframes

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Optional, Union
import sys
import time
import importlib
import sparse
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt
import os
#
sys.path.append("..")
import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)

In [3]:
## database connection parameters
# username = config.PG_USERNAME #we use peer authentication so don't need use vars, but in theory would pass them into config_path
# password = config.PG_PASSWORD
database_name = config.DB_NAME
print(database_name)
config_path = 'postgresql://{database_name}'.format(
    database_name = database_name
)
connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
schema_name = 'eol_test_ncjones' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")
# caching
reset_schema = False # if true, rebuild all data from scratch

# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)
# if reset_schema:
#     db.execute(
#         'drop schema if exists {} cascade'.format(schema_name)
#     )
# db.execute(
#     'create schema if not exists {}'.format(schema_name)
# )

localhost/omop_v6
cdm schema: cdm_6871_21


In [8]:
%%time
# Get the full condition item table
sql = """
    select
        *
    from
        {omop_schema}.{in_name} c
""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
    in_name=in_name
)
cohort = db.query(sql)
cohort = cohort.loc[cohort.antibiotic_type != 'inappropriate']

CPU times: user 8.78 s, sys: 434 ms, total: 9.21 s
Wall time: 11.7 s


### Viewing the antibiotic list and filtering by non first line, second line and alternatives

## Config and Helper Functions

In [9]:
group_to_cols = {"Arthritis" : ['arthritis_concept_ancestor',"arthritis"],
                 "Autoimmune" : ['autoimmune_concept_ancestor',"sjögren",  "rheumatoid_arthritis", "reactive_arthritis", "lupus_erythematosus", "dermatomyositis"],
                 "Cancer" : ['cancer_concept_ancestor',"cancer","carcinoma","malignan","leukemia","lymphoma","sarcoma"],
                 "Chronic Kidney" : ['chronic_kidney_concept_ancestor',"chronic_kidney","chronic_renal_failure"],
            "Diabetes Mellitus" : ['diabetes_mellitus_concept_ancestor',"diabetes_mellitus"], "Thyroid Disorder" : ['thyroid_disorder_concept_ancestor',"hashimoto_thyroiditis", "graves"], "HIV" : ['hiv_concept_ancestor',"hiv"], "Hypertension": ['hypertension_concept_ancestor',"hypertension"], "Menopause":['menopause_concept_ancestor',"menopause"]}

year_buckets = {"UTI_2012_14":[2012,2013,2014],"UTI_2015_17":[2015,2016,2017],"UTI_2018_21":[2018,2019,2020,2021]}

for group_time, years in year_buckets.items():
    cohort[group_time] = 1*(cohort.condition_start_date.dt.year.isin(years))
    
specialties = ['specialty_family_medicine_group',
 'specialty_internal_medicine_group',
 'specialty_emergency/acute_group',
 'specialty_advanced_specialist_group',
 'specialty_OBGYN_group',
 'specialty_other_group',
 'specialty_urology_group']

specialty_groups = {' '.join(x.replace("specialty_","").replace("_group","").split("_")) : [x] for x in specialties}


cohort["uti_1_year"] = 1*((cohort.days_since_previous_uti <= 365) & (cohort.days_since_previous_uti > 0))


In [10]:
for condition_cat, condition_cols in group_to_cols.items():
    if condition_cat == 'Menopause':
        group_to_cols[condition_cat] = ['menopause_any']
    else:
        group_to_cols[condition_cat] = [x + '1_2_yr' for x in condition_cols] #to convert back to the nomenclature in the pd dataframe getting all events:
        group_to_cols[condition_cat] += [x + '0_6_months' for x in condition_cols]
        group_to_cols[condition_cat] += [x + '6_months_1_yr' for x in condition_cols]
group_to_cols.update({"UTI history in 1 year" : ["uti_1_year"], "Fever at presentation" : ['fever'], "Urinalysis ordered" : ['urine_test_present'], 'Blood test ordered' : ['cbc_present'], "Last UTI in 2012-2014":["UTI_2012_14"], "Last UTI in 2015-2017":["UTI_2015_17"], "Last UTI in 2018-2021":["UTI_2018_21"]})

group_to_cols.update(specialty_groups)
ages = cohort.age.dropna()

In [11]:
group_to_cols

{'Arthritis': ['arthritis_concept_ancestor1_2_yr',
  'arthritis1_2_yr',
  'arthritis_concept_ancestor0_6_months',
  'arthritis0_6_months',
  'arthritis_concept_ancestor6_months_1_yr',
  'arthritis6_months_1_yr'],
 'Autoimmune': ['autoimmune_concept_ancestor1_2_yr',
  'sjögren1_2_yr',
  'rheumatoid_arthritis1_2_yr',
  'reactive_arthritis1_2_yr',
  'lupus_erythematosus1_2_yr',
  'dermatomyositis1_2_yr',
  'autoimmune_concept_ancestor0_6_months',
  'sjögren0_6_months',
  'rheumatoid_arthritis0_6_months',
  'reactive_arthritis0_6_months',
  'lupus_erythematosus0_6_months',
  'dermatomyositis0_6_months',
  'autoimmune_concept_ancestor6_months_1_yr',
  'sjögren6_months_1_yr',
  'rheumatoid_arthritis6_months_1_yr',
  'reactive_arthritis6_months_1_yr',
  'lupus_erythematosus6_months_1_yr',
  'dermatomyositis6_months_1_yr'],
 'Cancer': ['cancer_concept_ancestor1_2_yr',
  'cancer1_2_yr',
  'carcinoma1_2_yr',
  'malignan1_2_yr',
  'leukemia1_2_yr',
  'lymphoma1_2_yr',
  'sarcoma1_2_yr',
  'cancer

In [12]:
def compute_condition_metrics(df : pd.DataFrame ,name_dict : Dict[str, List[str]] =group_to_cols) -> Dict[str, List[Union[int,np.ndarray]]]:
    '''Computes the mean of a condition column and computes the standard deviation'''
    new_df = df.copy()
    means = []
    stdevs = []
    counts = []
    group_names = []
    for group_name, group_items in name_dict.items():
        new_df["temp_series"] = np.where(df[group_items].sum(axis=1) > 0, 1, 0)
        mean, std, = np.mean(new_df["temp_series"]), np.std(new_df["temp_series"])
        means.append(mean)
        stdevs.append(std)
        counts.append(len(new_df.loc[new_df["temp_series"] == 1]))
        group_names.append(group_name)
    return {"Condition Category" : group_names, "Mean" : means, "Standard Deviation" : stdevs, "Agg_count" : len(new_df), "Counts" : counts}


def get_columns_of_interest(df : pd.DataFrame ,name_dict : Dict[str, List[str]] =group_to_cols) -> Dict[str, List[Union[int,np.ndarray]]]:
    '''Computes the mean of a condition column and computes the standard deviation'''
    new_df = df.copy()
    group_names = []
    for group_name, group_items in name_dict.items():
        new_df[group_name.upper()] = np.where(df[group_items].sum(axis=1) > 0, 1, 0)
        group_names.append(group_name)
    subset_columns = [x.upper() for x in group_names] + ['antibiotic_type']
    subset_df = new_df[subset_columns]
    return subset_df

def compute_age_metrics(ages: pd.Series) -> Dict[str, List[Union[int,np.ndarray]]]:
    
                                                       
                                                       
    # First quartile (Q1)
    Q1 = np.percentile(ages, 25, interpolation ='midpoint')
  
    # Third quartile (Q3)
    Q3 = np.percentile(ages, 75, interpolation ='midpoint')

    # Interquaritle range (IQR)
    IQR = Q3 - Q1
    
    age_metrics = {'Max' : [np.max(ages)],
    'Min' : [np.min(ages)],
    'Mean' : [np.mean(ages)],
    'Median' : [np.median(ages)],
    'Standard Deviation': [np.std(ages)],
    'IQR' : [IQR]}


    return age_metrics

def to_df(data : Dict[str,List[object]]) -> pd.DataFrame :
    return pd.DataFrame(data=data)
                                 
def log_csv_table(data : Dict[str,List[object]],name : str ="table_1_agg_conditions") -> None:
    df = to_df(data=data)
    return df.to_csv(f"{name}.csv",index=False)


## Condition and Age Information

In [17]:
# #loop over modified dataframes that include a single condition and append it to dictionary
atype_to_metrics = {'all_antibiotics': [cohort.antibiotic_type.value_counts().index.tolist(),None],'first_line': [['nitrofurantoin','trimethoprim-sulfamethoxazole'],None],'second_line':[['second_line'],None],'alternatives':[['alternatives'],None]}

condition_table_name = f'{out_name}_conditions'
age_table_name = f'{out_name}_age'

log_path = 'Logs_2023/excel_table1_v2/'
if not os.path.exists(log_path):
    os.makedirs(log_path)

condition_writer = pd.ExcelWriter(log_path + condition_table_name + '.xlsx', engine = 'xlsxwriter')
age_writer = pd.ExcelWriter(log_path + age_table_name + '.xlsx', engine = 'xlsxwriter')

for a_type, a_names in atype_to_metrics.items():
    temp_c_metrics = compute_condition_metrics(cohort.loc[cohort.antibiotic_type.isin(a_names[0])])
#     atype_to_metrics[a_type][1] = temp_c_metrics
    to_df(temp_c_metrics).to_excel(condition_writer, sheet_name = a_type)
#     log_csv_table(temp_metrics, f"table_1_{a_type}_conditions")

    temp_a_metrics = compute_age_metrics(cohort.loc[cohort.antibiotic_type.isin(a_names[0])].age)
    to_df(temp_a_metrics).to_excel(age_writer, sheet_name = a_type)


condition_writer.save()
condition_writer.close()
age_writer.save()
age_writer.close()


/home/ncjones/miniconda3/envs/omop/lib/python3.8/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### Compute P Values

#### Age

In [5]:
import pandas as pd
import scipy 
import numpy as np

np.random.seed(seed=42)
for column in ['second_line','alternatives']:
    print(f"P value for ages of {column} vs. first line")
    firstline_df = cohort.loc[cohort.antibiotic_type.isin(['nitrofurantoin','trimethoprim-sulfamethoxazole'])]
    query_df = cohort.loc[cohort.antibiotic_type.isin([column])]
    p_value = scipy.stats.ttest_ind(firstline_df['age'].values,query_df['age'].values)[1]
    print(p_value)

P value for ages of second_line vs. first line
0.0
P value for ages of alternatives vs. first line
3.719370305379694e-76


#### Conditions

In [16]:

from collections import defaultdict
import scipy
import numpy as np
from pprint import pprint

np.random.seed(seed=42)
#initialize empty dictionary
comparison_to_p_value = dict()
temp_df = cohort.copy()
for group, _ in group_to_cols.items():
    new_group = group.upper()
    temp_df[new_group] = np.where(temp_df[group_to_cols[group]].sum(axis=1) > 0, 1, 0)

    
atypes_to_cols = {'first_line': ['nitrofurantoin','trimethoprim-sulfamethoxazole'],'second_line':['second_line'],'alternatives':['alternatives']}

#for second line and alternatives dfs
for condition_query in ['MENOPAUSE','UTI HISTORY IN 1 YEAR','HYPERTENSION','DIABETES MELLITUS','ARTHRITIS','CANCER','CHRONIC KIDNEY','AUTOIMMUNE','THYROID DISORDER']:
    for atype in ['second_line','alternatives']:
        first_line_df = temp_df.loc[temp_df.antibiotic_type.isin(atypes_to_cols['first_line'])]
        atype_df = temp_df.loc[temp_df.antibiotic_type.isin(atypes_to_cols[atype])]
        #for every single condition
        #populate dictionary with ttest comparison and p value
        ttest_name = f"{condition_query}_{atype}"
        comparison_to_p_value[ttest_name] = scipy.stats.ttest_ind(first_line_df[condition_query].values,atype_df[condition_query].values)[1]

#print out the dictionary
pprint(comparison_to_p_value,sort_dicts=False)

{'MENOPAUSE_second_line': 0.6098119755188777,
 'MENOPAUSE_alternatives': 0.1613656357785185,
 'UTI HISTORY IN 1 YEAR_second_line': 0.00048574313781707976,
 'UTI HISTORY IN 1 YEAR_alternatives': 2.2962855782770367e-05,
 'HYPERTENSION_second_line': 9.259612766332118e-262,
 'HYPERTENSION_alternatives': 2.9806212100994945e-72,
 'DIABETES MELLITUS_second_line': 4.4183869145111045e-115,
 'DIABETES MELLITUS_alternatives': 2.049893693250875e-40,
 'ARTHRITIS_second_line': 2.1417986539918084e-43,
 'ARTHRITIS_alternatives': 2.0486281851173674e-24,
 'CANCER_second_line': 9.482369003912521e-64,
 'CANCER_alternatives': 1.4618679661388294e-14,
 'CHRONIC KIDNEY_second_line': 3.4848258316875044e-56,
 'CHRONIC KIDNEY_alternatives': 1.3080775289957286e-49,
 'AUTOIMMUNE_second_line': 7.14789081185416e-08,
 'AUTOIMMUNE_alternatives': 1.550206997486111e-07,
 'THYROID DISORDER_second_line': 0.0003539660377743022,
 'THYROID DISORDER_alternatives': 0.4338829155160906}


In [21]:

from collections import defaultdict
import scipy
import numpy as np
from pprint import pprint

np.random.seed(seed=42)
#initialize empty dictionary
comparison_to_p_value = dict()
temp_df = cohort.copy()
for group, _ in group_to_cols.items():
    new_group = group.upper()
    temp_df[new_group] = np.where(temp_df[group_to_cols[group]].sum(axis=1) > 0, 1, 0)

    
atypes_to_cols = {'first_line': ['nitrofurantoin','trimethoprim-sulfamethoxazole'],'second_line':['second_line'],'alternatives':['alternatives']}

condition_queries = ['MENOPAUSE','UTI HISTORY IN 1 YEAR','HYPERTENSION','DIABETES MELLITUS','ARTHRITIS','CANCER','CHRONIC KIDNEY','AUTOIMMUNE','THYROID DISORDER']
other_queries = [x.upper() for x in ['Fever at presentation', 'Urinalysis ordered','Blood test ordered','Last UTI in 2012-2014','Last UTI in 2015-2017','Last UTI in 2018-2021','family medicine','internal medicine','emergency/acute','advanced specialist','OBGYN','other','urology']]
all_queries = condition_queries + other_queries + ['age']
#for second line and alternatives dfs
for condition_query in all_queries:
    for atype in ['second_line','alternatives']:
        first_line_df = temp_df.loc[temp_df.antibiotic_type.isin(atypes_to_cols['first_line'])]
        atype_df = temp_df.loc[temp_df.antibiotic_type.isin(atypes_to_cols[atype])]
        #for every single condition
        #populate dictionary with ttest comparison and p value
        ttest_name = f"{condition_query}_{atype}"
        comparison_to_p_value[ttest_name] = scipy.stats.ttest_ind(first_line_df[condition_query].values,atype_df[condition_query].values)[1]

#print out the dictionary
pprint(comparison_to_p_value,sort_dicts=False)

{'MENOPAUSE_second_line': 0.6098119755188777,
 'MENOPAUSE_alternatives': 0.1613656357785185,
 'UTI HISTORY IN 1 YEAR_second_line': 0.00048574313781707976,
 'UTI HISTORY IN 1 YEAR_alternatives': 2.2962855782770367e-05,
 'HYPERTENSION_second_line': 9.259612766332118e-262,
 'HYPERTENSION_alternatives': 2.9806212100994945e-72,
 'DIABETES MELLITUS_second_line': 4.4183869145111045e-115,
 'DIABETES MELLITUS_alternatives': 2.049893693250875e-40,
 'ARTHRITIS_second_line': 2.1417986539918084e-43,
 'ARTHRITIS_alternatives': 2.0486281851173674e-24,
 'CANCER_second_line': 9.482369003912521e-64,
 'CANCER_alternatives': 1.4618679661388294e-14,
 'CHRONIC KIDNEY_second_line': 3.4848258316875044e-56,
 'CHRONIC KIDNEY_alternatives': 1.3080775289957286e-49,
 'AUTOIMMUNE_second_line': 7.14789081185416e-08,
 'AUTOIMMUNE_alternatives': 1.550206997486111e-07,
 'THYROID DISORDER_second_line': 0.0003539660377743022,
 'THYROID DISORDER_alternatives': 0.4338829155160906,
 'FEVER AT PRESENTATION_second_line': 7.96

In [36]:
def remove_group_tags(x):
    
    x = x.replace('_second_line','')
    x = x.replace('_alternatives','')
    return x

p_value_dataframe = pd.DataFrame([comparison_to_p_value]).T.reset_index()
p_value_dataframe.columns = ["comparison","p_value"]
second_line_ps = p_value_dataframe.loc[p_value_dataframe.comparison.str.endswith('second_line'),'p_value']
alternatives_ps = p_value_dataframe.loc[p_value_dataframe.comparison.str.endswith('alternatives'),'p_value']
p_value_dataframe['comparison'] = p_value_dataframe.comparison.apply(lambda x: remove_group_tags(x))
p_value_dataframe.drop_duplicates(subset=['comparison'],inplace=True)
p_value_dataframe.drop(columns=['p_value'],inplace=True)
p_value_dataframe['second_line_pval'] = second_line_ps.values
p_value_dataframe['alternatives_pval'] = alternatives_ps.values

In [37]:
p_value_dataframe.to_csv(f"{out_name}_pvalues.csv")

In [31]:
#OLD FOR COMPARISON
from collections import defaultdict
import scipy
import numpy as np
from pprint import pprint

np.random.seed(seed=42)
#initialize empty dictionary
comparison_to_p_value = dict()
temp_df = cohort.copy()
for group, _ in group_to_cols.items():
    new_group = group.upper()
    temp_df[new_group] = np.where(temp_df[group_to_cols[group]].sum(axis=1) > 0, 1, 0)

    
atypes_to_cols = {'first_line': ['nitrofurantoin','trimethoprim-sulfamethoxazole'],'second_line':['second_line'],'alternatives':['alternatives']}

#for second line and alternatives dfs
for condition_query in ['MENOPAUSE','UTI HISTORY IN 1 YEAR','HYPERTENSION','DIABETES MELLITUS','ARTHRITIS','CANCER','CHRONIC KIDNEY','AUTOIMMUNE','THYROID DISORDER']:
    for atype in ['second_line','alternatives']:
        first_line_df = temp_df.loc[temp_df.antibiotic_type.isin(atypes_to_cols['first_line'])]
        atype_df = temp_df.loc[temp_df.antibiotic_type.isin(atypes_to_cols[atype])]
        #for every single condition
        #populate ditionary with ttest comparison and p value
        ttest_name = f"{condition_query}_{atype}"
        comparison_to_p_value[ttest_name] = scipy.stats.ttest_ind(first_line_df[condition_query].values,atype_df[condition_query].values)[1]

#print out the dictionary
pprint(comparison_to_p_value,sort_dicts=False)

{'MENOPAUSE_second_line': 0.6599642409074162,
 'MENOPAUSE_alternatives': 0.04808063076941972,
 'UTI HISTORY IN 1 YEAR_second_line': 0.000826890022034254,
 'UTI HISTORY IN 1 YEAR_alternatives': 0.07016806719316435,
 'HYPERTENSION_second_line': 0.0,
 'HYPERTENSION_alternatives': 0.0,
 'DIABETES MELLITUS_second_line': 1.4318319287329086e-144,
 'DIABETES MELLITUS_alternatives': 1.3600761366184522e-169,
 'ARTHRITIS_second_line': 7.734879081226961e-67,
 'ARTHRITIS_alternatives': 9.844564950352883e-118,
 'CANCER_second_line': 2.611074927986663e-72,
 'CANCER_alternatives': 3.112595688875892e-69,
 'CHRONIC KIDNEY_second_line': 4.0567016791720076e-92,
 'CHRONIC KIDNEY_alternatives': 6.391874985777473e-221,
 'AUTOIMMUNE_second_line': 3.3277697198034643e-08,
 'AUTOIMMUNE_alternatives': 7.368249948563195e-19,
 'THYROID DISORDER_second_line': 0.01058280203805486,
 'THYROID DISORDER_alternatives': 0.9374614901266345}
